In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

import hanja
from hanja import hangul

from kiwipiepy import Kiwi
from tqdm import tqdm
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer


In [2]:
df = pd.read_csv("/data/ephemeral/home/yujin/yongari_dataset/processed_train_4220.csv")
df = df.dropna(subset=['text'])

In [7]:
grouped_df = df.groupby('target')

In [8]:
# 각 그룹별로 데이터프레임을 저장
target_0_df = grouped_df.get_group(0)  # target이 0인 데이터
target_1_df = grouped_df.get_group(1)  # target이 1인 데이터
target_2_df = grouped_df.get_group(2)  # target이 2인 데이터
target_3_df = grouped_df.get_group(3)  # target이 3인 데이터
target_4_df = grouped_df.get_group(4)  # target이 4인 데이터
target_5_df = grouped_df.get_group(5)  # target이 5인 데이터
target_6_df = grouped_df.get_group(6)  # target이 6인 데이터

In [3]:
kiwi = Kiwi()

In [13]:
# Stopwords 설정
stopwords = ['종합']

# 특수기호와 변환할 문자열 매핑
replacement_dict = {
    'U＋': '유플러스',
    '%': '퍼센트',
    '％': '퍼센트',
    '…': ' ',
    '.': '-',
    '~': '-',
    '..': '-',
    '...': ' ',
    '"': '-',
    '“': '"',
    '”': '"',
    '’': ' ',
    "'": '-',
    '·': '-',
    '↔': '-',
    '㈜': '-',
    '→': '에서',
    '↑': '상승',
    '↓': '하락',
    'ｍ': '미터',
    '㎝': '센티미터',
    '㎜': '밀리미터',
    '㎛': '마이크로미터',
    '㎞': '킬로미터',
    '㎡': '제곱미터',
    '＋': '플러스',
    '㎏': '킬로그램',
    '℃': '도',
    '㎓': '기가헤르츠'
}

In [14]:
# 데이터 전처리
def preprocess_text(text):
    """
    텍스트 전처리 함수 (한자 및 특수기호 처리)

    Args:
        text: 전처리할 텍스트

    Returns:
        processed_text: 전처리된 텍스트
    """
    # 특수기호 치환
    for char, replacement in replacement_dict.items():
        text = text.replace(char, replacement)
    
    # 특수기호 제거 (한글, 영어, 숫자, 공백만 남기기)
    text = re.sub(r'[^가-힣a-zA-Z0-9\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    
    # 한자 한글로 치환
    text = hanja.translate(text, 'substitution')
    
    return text

In [15]:
# 품사 분류
def part_classification(text):
    """
    텍스트의 품사를 태깅(분류)하는 함수

    Args:
        text: 데이터셋의 'text'열의 데이터

    Returns:
        raw_part_class: 각 텍스트의 토큰에 대한 품사 정보
    """
    raw_part_class = kiwi.tokenize(text, normalize_coda=False)
    return raw_part_class

In [16]:
# 품사 필터링 (명사, 동사, 형용사, 어근)
MAIN_PART = ['NNG', 'NNB', 'NNP', 'VV', 'VA', 'XR'] # 필터링할 주요 품사
VERB_ADJ_PART = ['VV', 'VA'] # 동사, 형용사

def part_filtering(df, text_col):
    """
    데이터프레임의 각 텍스트에서 필터링된 단어 리스트를 생성하는 함수

    Args:
        df: 필터링 할 대상 데이터프레임
        text_col: 각 토큰에 대해 태그된 품사 정보 열

    Returns:
        filtered_words: 필터링된 단어 리스트
    """
    filtered_words = []

    for index, row in tqdm(df.iterrows(), total=df.shape[0]):
        if row[text_col]:
            class_filter = [
                token.form + '다' if token.tag in VERB_ADJ_PART else token.form
                for token in row[text_col]
                if token.tag in MAIN_PART and len(token.form) > 1 and token.form not in stopwords
            ]
            filtered_words.append(class_filter)
        else:
            filtered_words.append([])

    return filtered_words

In [17]:
# 단어 개수 세기
def word_count(word_list):
    """
    단어 리스트에서 단어의 개수를 세고 정렬하여 반환하는 함수

    Args:
        word_list: 필터링된 단어 리스트 열

    Returns:
        sorted_words: 개수가 많은 순서대로 정렬된 단어 리스트
    """
    word_counter = Counter(word_list)
    sorted_words = sorted(word_counter.items(), key=lambda item: item[1], reverse=True)

    return sorted_words

In [18]:
df['cleaned_text'] = df['text'].apply(preprocess_text)

In [19]:
df['text_class'] = df['cleaned_text'].apply(part_classification)

In [20]:
df['text_class_filter'] = part_filtering(df, 'text_class')

100%|██████████| 4219/4219 [00:00<00:00, 17748.80it/s]


In [21]:
df

,ID,text,target,cleaned_text,text_class,text_class_filter
0,aug0,'두산밥캣 맥스 컨트롤 2019년 혁신제품으로 골드상을 수상하였다.',5,두산밥캣 맥스 컨트롤 2019년 혁신제품으로 골드상을 수상하였다,"[(두산, NNP, 0, 2), (밥캣, NNP, 2, 2), (맥스, NNP, 5...","[두산, 밥캣, 맥스, 컨트롤, 혁신, 제품, 골드, 수상]"
1,aug1,'삼성전자 특징주가 엿새 만에 약세로 전환되었습니다.',5,삼성전자 특징주가 엿새 만에 약세로 전환되었습니다,"[(삼성전자, NNP, 0, 4), (특징, NNG, 5, 2), (주, NNG, ...","[삼성전자, 특징, 엿새, 약세, 전환]"
2,aug2,손흥민은 E2트리 A컵에서 강서U3일리스트 2번을 선택하겠다.,1,손흥민은 E2트리 A컵에서 강서U3일리스트 2번을 선택하겠다,"[(손흥민, NNP, 0, 3), (은, JX, 3, 1), (E, SL, 5, 1...","[손흥민, 트리, 강서, 리스트, 선택]"
3,aug3,'아르헨티나가 리오�르스를 5-1로 이기며 2위전인 링컨시티와 맞붙게 된다.',1,아르헨티나가 리오 르스를 5 1로 이기며 2위전인 링컨시티와 맞붙게 된다,"[(아르헨티나, NNP, 0, 5), (가, JKS, 5, 1), (리오, NNP,...","[아르헨티나, 리오, 르스를, 이기다, 링컨시티, 맞붙다]"
4,aug4,'네이버 유튜버의 지하철 2회 방송은 문화체육부로부터 2편 모두 호평을 받았다.',4,네이버 유튜버의 지하철 2회 방송은 문화체육부로부터 2편 모두 호평을 받았다,"[(네이버, NNP, 0, 3), (유튜버, NNP, 4, 3), (의, JKG, ...","[네이버, 유튜버, 지하철, 방송, 문화, 체육부, 호평]"
...,...,...,...,...,...,...
4214,ynat-v1_train_02795,트럼프 폭스뉴스 앵커들 충성도 점수매겨…10점만점에 12점도,6,트럼프 폭스뉴스 앵커들 충성도 점수매겨 10점만점에 12점도,"[(트럼프, NNP, 0, 3), (폭스뉴스, NNP, 4, 4), (앵커, NNG...","[트럼프, 폭스뉴스, 앵커, 충성도, 점수, 매기다, 만점]"
4215,ynat-v1_train_02796,삼성 갤럭시S9 정식 출시 첫 주말 이통시장 잠잠,4,삼성 갤럭시S9 정식 출시 첫 주말 이통시장 잠잠,"[(삼성, NNP, 0, 2), (갤럭시S, NNP, 3, 4), (9, SN, 7...","[삼성, 갤럭시S, 정식, 출시, 주말, 이통, 시장, 잠자다]"
4216,ynat-v1_train_02797,텔레그램 한인 등 아시아서 2시간 다운…서비스 정상화종료2보,4,텔레그램 한인 등 아시아서 2시간 다운 서비스 정상화종료2보,"[(텔레그램, NNP, 0, 4), (한인, NNG, 5, 2), (등, NNB, ...","[텔레그램, 한인, 아시아, 시간, 서비스, 정상, 종료]"
4217,ynat-v1_train_02798,인터뷰 류현진 친구에게 안타 맞는 것 싫어해…승부는 냉정,1,인터뷰 류현진 친구에게 안타 맞는 것 싫어해 승부는 냉정,"[(인터뷰, NNG, 0, 3), (류현진, NNP, 4, 3), (친구, NNP,...","[인터뷰, 류현진, 친구, 안타, 싫어하다, 승부, 냉정]"


In [22]:
df_cnt = [word for sublist in df['text_class_filter'] for word in sublist]
word_counts = word_count(df_cnt)
word_counts[:30]

[('대통령', 166),
 ('퍼센트', 165),
 ('한국', 118),
 ('대하다', 103),
 ('프로', 101),
 ('미국', 91),
 ('북한', 83),
 ('지원', 82),
 ('이란', 80),
 ('위하다', 74),
 ('개발', 70),
 ('대표', 68),
 ('출시', 67),
 ('농구', 65),
 ('세계', 64),
 ('시즌', 63),
 ('시장', 60),
 ('중국', 59),
 ('상승', 58),
 ('시작', 57),
 ('축구', 57),
 ('기술', 57),
 ('투자', 55),
 ('서울', 52),
 ('트럼프', 51),
 ('증가', 51),
 ('삼성', 50),
 ('정부', 50),
 ('경기', 50),
 ('정상', 49)]

In [23]:
len(df_cnt)

26502

### (1) Target 0 (생활문화)

In [24]:
target_0_df['cleaned_text'] = target_0_df['text'].apply(preprocess_text)
target_0_df['text_class'] = target_0_df['cleaned_text'].apply(part_classification)
target_0_df['text_class_filter'] = part_filtering(target_0_df, 'text_class')
target_0_df

/tmp/ipykernel_364431/2536432609.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_0_df['cleaned_text'] = target_0_df['text'].apply(preprocess_text)
/tmp/ipykernel_364431/2536432609.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_0_df['text_class'] = target_0_df['cleaned_text'].apply(part_classification)
100%|██████████| 605/605 [00:00<00:00, 17752.83it/s]
/tmp/ipykernel_364431/2536432609.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Tr

,ID,text,target,cleaned_text,text_class,text_class_filter
6,aug6,'4월 25일부터 전국에 고기압이 도달한 후기에 소나기는 가끔 나타난다.',0,4월 25일부터 전국에 고기압이 도달한 후기에 소나기는 가끔 나타난다,"[(4, SN, 0, 1), (월, NNB, 1, 1), (25, SN, 3, 2)...","[전국, 고기압, 도달, 후기, 소나기, 나타나다]"
10,aug10,한국에서 열리는 일본 문화 축제와 코믹콘2.,0,한국에서 열리는 일본 문화 축제와 코믹콘2,"[(한국, NNP, 0, 2), (에서, JKB, 2, 2), (열리, VV, 5,...","[한국, 열리다, 일본, 문화, 축제, 코믹]"
18,aug18,펭귄의 신기한 사생감과 완벽한 조각처럼 사자와 같은 모습을 한 펭귄.,0,펭귄의 신기한 사생감과 완벽한 조각처럼 사자와 같은 모습을 한 펭귄,"[(펭귄, NNG, 0, 2), (의, JKG, 2, 1), (신기, XR, 4, ...","[펭귄, 신기, 사생, 완벽, 조각, 사자, 모습, 펭귄]"
21,aug21,롯데백화점은 퍼펙트 웨딩쇼를 진행합니다.,0,롯데백화점은 퍼펙트 웨딩쇼를 진행합니다,"[(롯데백화점, NNP, 0, 5), (은, JX, 5, 1), (퍼펙트 웨딩, N...","[롯데백화점, 퍼펙트 웨딩, 진행]"
30,aug30,'3월 31일부터 3월 31일까지는 절반.',0,3월 31일부터 3월 31일까지는 절반,"[(3, SN, 0, 1), (월, NNB, 1, 1), (31, SN, 3, 2)...",[절반]
...,...,...,...,...,...,...
4177,ynat-v1_train_02753,진주 백백숲 걷기·힐링 스트레스 가꾼다,0,진주 백백숲 걷기 힐링 스트레스 가꾼다,"[(진주, NNG, 0, 2), (백, NR, 3, 1), (백, NR, 4, 1)...","[진주, 걷기, 힐링, 스트레스, 가꾸다]"
4190,ynat-v1_train_02767,청년한류 관광전국 큰일 한 번에…감동의 여행하세요,0,청년한류 관광전국 큰일 한 번에 감동의 여행하세요,"[(청년, NNG, 0, 2), (한류, NNG, 2, 2), (관광, NNG, 5...","[청년, 한류, 관광, 전국, 큰일, 감동, 여행]"
4198,ynat-v1_train_02777,금강 따라 시간여행 떠나요…공주 석장리 구석기 축제 개막,0,금강 따라 시간여행 떠나요 공주 석장리 구석기 축제 개막,"[(금강, NNP, 0, 2), (따르, VV, 3, 2), (어, EC, 4, 1...","[금강, 따르다, 시간, 여행, 떠나다, 공주, 석장리, 구석기, 축제, 개막]"
4206,ynat-v1_train_02786,텅 빈 교실 선생님의홀에서 연극으로…전북서 첫 공연,0,텅 빈 교실 선생님의홀에서 연극으로 전북서 첫 공연,"[(텅, MAG, 0, 1), (비, VV, 2, 1), (ᆫ, ETM, 2, 1)...","[교실, 선생, 연극, 전북, 공연]"


In [25]:
target_0_df_cnt = list(target_0_df.loc[:, 'text_class_filter'])
target_0_df_cnt = [data for inner_list in target_0_df_cnt for data in inner_list]
target_0_df_cnt = word_count(target_0_df_cnt)
target_0_df_cnt[:30]

[('여행', 28),
 ('축제', 26),
 ('날씨', 24),
 ('신간', 20),
 ('제주', 20),
 ('한국', 19),
 ('전국', 17),
 ('공연', 17),
 ('연주', 16),
 ('시간', 16),
 ('지역', 15),
 ('주의보', 15),
 ('남쪽', 15),
 ('강원', 14),
 ('주말', 14),
 ('서울', 13),
 ('동안', 13),
 ('개막', 13),
 ('예술', 13),
 ('걷기', 13),
 ('구름', 13),
 ('충북', 12),
 ('즐기다', 12),
 ('먼지', 12),
 ('열리다', 11),
 ('영화', 11),
 ('국제', 11),
 ('미세', 11),
 ('문화', 10),
 ('부산', 10)]

In [26]:
len(target_0_df_cnt)

1541

In [ ]:
target_0_df = pd.DataFrame(target_0_df_cnt,columns=['word','cnt'])
target_0_df

,word,cnt
0,여행,28
1,축제,26
2,날씨,24
3,신간,20
4,제주,20
...,...,...
1536,감동,1
1537,금강,1
1538,공주,1
1539,석장리,1


In [28]:
target_0_drop_df = target_0_df[target_0_df['cnt']>9]
target_0_drop_df

,word,cnt
0,여행,28
1,축제,26
2,날씨,24
3,신간,20
4,제주,20
5,한국,19
6,전국,17
7,공연,17
8,연주,16
9,시간,16


In [41]:
target_0_df.to_csv('target_0_df.csv', encoding='utf-8-sig', index=False)
target_0_drop_df.to_csv('target_0_drop_df.csv', encoding='utf-8-sig', index=False)

### (2) Target 1 (스포츠)

In [30]:
target_1_df['cleaned_text'] = target_1_df['text'].apply(preprocess_text)
target_1_df['text_class'] = target_1_df['cleaned_text'].apply(part_classification)
target_1_df['text_class_filter'] = part_filtering(target_1_df, 'text_class')
target_1_df

/tmp/ipykernel_364431/2366291003.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_1_df['cleaned_text'] = target_1_df['text'].apply(preprocess_text)
/tmp/ipykernel_364431/2366291003.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_1_df['text_class'] = target_1_df['cleaned_text'].apply(part_classification)
100%|██████████| 614/614 [00:00<00:00, 17424.83it/s]
/tmp/ipykernel_364431/2366291003.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Tr

,ID,text,target,cleaned_text,text_class,text_class_filter
2,aug2,손흥민은 E2트리 A컵에서 강서U3일리스트 2번을 선택하겠다.,1,손흥민은 E2트리 A컵에서 강서U3일리스트 2번을 선택하겠다,"[(손흥민, NNP, 0, 3), (은, JX, 3, 1), (E, SL, 5, 1...","[손흥민, 트리, 강서, 리스트, 선택]"
3,aug3,'아르헨티나가 리오�르스를 5-1로 이기며 2위전인 링컨시티와 맞붙게 된다.',1,아르헨티나가 리오 르스를 5 1로 이기며 2위전인 링컨시티와 맞붙게 된다,"[(아르헨티나, NNP, 0, 5), (가, JKS, 5, 1), (리오, NNP,...","[아르헨티나, 리오, 르스를, 이기다, 링컨시티, 맞붙다]"
7,aug7,'베르만이 한국으로 복귀하며 2년 만에 한국 수비손으로 돌아왔다.',1,베르만이 한국으로 복귀하며 2년 만에 한국 수비손으로 돌아왔다,"[(베르, NNG, 0, 2), (만, JX, 2, 1), (이, JKS, 3, 1...","[베르, 한국, 복귀, 한국, 수비, 돌아오다]"
14,aug14,'캄보디아 샌파이어 아시아 챔피언십에서 1위를 차지한 마피는 72시간 동안 납치된 ...,1,캄보디아 샌파이어 아시아 챔피언십에서 1위를 차지한 마피는 72시간 동안 납치된 사...,"[(캄보디아, NNP, 0, 4), (샌, NNP, 5, 1), (파이어, NNG,...","[캄보디아, 파이어, 아시아, 챔피언십, 차지, 마피, 시간, 동안, 납치, 사실,..."
17,aug17,류현진이 27일 서울에서 팬미팅을 열어 새 시즌 준비를 알린다.,1,류현진이 27일 서울에서 팬미팅을 열어 새 시즌 준비를 알린다,"[(류현진, NNP, 0, 3), (이, JKS, 3, 1), (27, SN, 5,...","[류현진, 서울, 미팅, 시즌, 준비, 알리다]"
...,...,...,...,...,...,...
4187,ynat-v1_train_02763,12월부터 손흥민 새로운 시즌보다 가속도,1,12월부터 손흥민 새로운 시즌보다 가속도,"[(12, SN, 0, 2), (월, NNB, 2, 1), (부터, JX, 3, 2...","[손흥민, 시즌, 가속도]"
4188,ynat-v1_train_02764,美 프로야구 주말대서 충격…8명 부상…1명 중상2보,1,프로야구 주말대서 충격 8명 부상 1명 중상2보,"[(프로, NNG, 0, 2), (야구, NNG, 2, 2), (주말, NNG, 5...","[프로, 야구, 주말, 충격, 부상, 중상]"
4203,ynat-v1_train_02782,김현진의 다방면 무난 빠른 템포 팀 성적까지 영역,1,김현진의 다방면 무난 빠른 템포 팀 성적까지 영역,"[(김현진, NNP, 0, 3), (의, JKG, 3, 1), (다방면, NNG, ...","[김현진, 다방면, 빠르다, 템포, 성적, 영역]"
4210,ynat-v1_train_02790,프로농구 개막 ③ 헤인즈의 1만점·양동근의 900스틸…종합,1,프로농구 개막 헤인즈의 1만점 양동근의 900스틸 종합,"[(프로, NNG, 0, 2), (농구, NNG, 2, 2), (개막, NNG, 5...","[프로, 농구, 개막, 헤인즈, 양동근, 스틸]"


In [31]:
target_1_df_cnt = list(target_1_df.loc[:, 'text_class_filter'])
target_1_df_cnt = [data for inner_list in target_1_df_cnt for data in inner_list]
target_1_df_cnt = word_count(target_1_df_cnt)
print(len(target_1_df_cnt))
target_1_df_cnt[:30]

1601


[('프로', 93),
 ('농구', 64),
 ('시즌', 57),
 ('축구', 53),
 ('감독', 45),
 ('선수', 41),
 ('승리', 36),
 ('리그', 33),
 ('경기', 33),
 ('류현진', 28),
 ('한국', 27),
 ('대표', 26),
 ('연승', 26),
 ('야구', 25),
 ('연속', 25),
 ('게임', 24),
 ('월드컵', 23),
 ('아시안', 23),
 ('연패', 22),
 ('기록', 21),
 ('패배', 19),
 ('우승', 19),
 ('계약', 17),
 ('출전', 16),
 ('부상', 16),
 ('여자', 15),
 ('손흥민', 14),
 ('아시아', 14),
 ('이닝', 14),
 ('실점', 14)]

In [33]:
target_1_df = pd.DataFrame(target_1_df_cnt,columns=['word','cnt'])
target_1_df

,word,cnt
0,프로,93
1,농구,64
2,시즌,57
3,축구,53
4,감독,45
...,...,...
1596,영역,1
1597,양동근,1
1598,친구,1
1599,싫어하다,1


In [34]:
target_1_drop_df = target_1_df[target_1_df['cnt']>9]
target_1_drop_df

,word,cnt
0,프로,93
1,농구,64
2,시즌,57
3,축구,53
4,감독,45
5,선수,41
6,승리,36
7,리그,33
8,경기,33
9,류현진,28


In [40]:
target_1_df.to_csv('target_1_df.csv', encoding='utf-8-sig', index=False)
target_1_drop_df.to_csv('target_1_drop_df.csv', encoding='utf-8-sig', index=False)

### (3) Target 2 (국내 정치)

In [35]:
target_2_df['cleaned_text'] = target_2_df['text'].apply(preprocess_text)
target_2_df['text_class'] = target_2_df['cleaned_text'].apply(part_classification)
target_2_df['text_class_filter'] = part_filtering(target_2_df, 'text_class')
target_2_df

/tmp/ipykernel_364431/4223245892.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_2_df['cleaned_text'] = target_2_df['text'].apply(preprocess_text)
/tmp/ipykernel_364431/4223245892.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_2_df['text_class'] = target_2_df['cleaned_text'].apply(part_classification)
100%|██████████| 587/587 [00:00<00:00, 17345.88it/s]
/tmp/ipykernel_364431/4223245892.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Tr

,ID,text,target,cleaned_text,text_class,text_class_filter
5,aug5,'현미경으로 송철호 울산 시장의 발언을 경청한다.',2,현미경으로 송철호 울산 시장의 발언을 경청한다,"[(현미경, NNG, 0, 3), (으로, JKB, 3, 2), (송철호, NNP,...","[현미경, 송철호, 울산, 시장, 발언, 경청]"
9,aug9,"'혁신형 관리형 중진들, 사대위 의견에 대해 팽청종합 의견을 제시했다.'",2,혁신형 관리형 중진들 사대위 의견에 대해 팽청종합 의견을 제시했다,"[(혁신, NNG, 0, 2), (형, XSN, 2, 1), (관리, NNG, 4,...","[혁신, 관리, 중진, 사대, 의견, 대하다, 의견, 제시]"
11,aug11,"트럼프가 자력으로 대선 후보가 되길 희망하며, 그의 전망은 불확실하다.",2,트럼프가 자력으로 대선 후보가 되길 희망하며 그의 전망은 불확실하다,"[(트럼프, NNP, 0, 3), (가, JKS, 3, 1), (자력, NNG, 5...","[트럼프, 자력, 대선, 후보, 희망, 전망, 확실]"
24,aug24,'한반도 평화를 논하기 위해 남북 북미 정상회담을 개최한다.',2,한반도 평화를 논하기 위해 남북 북미 정상회담을 개최한다,"[(한반도, NNP, 0, 3), (평화, NNG, 4, 2), (를, JKO, 6...","[한반도, 평화, 논하다, 위하다, 남북, 북미, 정상, 회담, 개최]"
25,aug25,북한 노동당이 5월 6일에 제7차 대회를 개최한다고 보도되었다.,2,북한 노동당이 5월 6일에 제7차 대회를 개최한다고 보도되었다,"[(북한, NNP, 0, 2), (노동당, NNP, 3, 3), (이, JKS, 6...","[북한, 노동당, 대회, 개최, 보도]"
...,...,...,...,...,...,...
4197,ynat-v1_train_02776,송희경 2호 이적출장 시리즈 등 아시안비즈니스스포츠 기자용,2,송희경 2호 이적출장 시리즈 등 아시안비즈니스스포츠 기자용,"[(송희경, NNP, 0, 3), (2, SN, 4, 1), (호, NNB, 5, ...","[송희경, 이적, 출장, 시리즈, 아시안, 비즈니스, 스포츠, 기자]"
4201,ynat-v1_train_02780,北 대북 김정은 향한 성경쟁 무대된 릴레이 토론,2,대북 김정은 향한 성경쟁 무대된 릴레이 토론,"[(대북, NNG, 0, 2), (김정은, NNP, 3, 3), (향하, VV, 7...","[대북, 김정은, 향하다, 경쟁, 무대, 릴레이, 토론]"
4205,ynat-v1_train_02785,민주화운동 보상받으면 국가배상 안 돼…헌재와 반대 판결,2,민주화운동 보상받으면 국가배상 안 돼 헌재와 반대 판결,"[(민주, NNG, 0, 2), (화, XSN, 2, 1), (운동, NNG, 3,...","[민주, 운동, 보상, 국가, 배상, 헌재, 반대, 판결]"
4209,ynat-v1_train_02789,이총리 세대4.5 직업 완전히 못해져할 일 꽤 남아,2,이총리 세대4 5 직업 완전히 못해져할 일 꽤 남아,"[(이, MM, 0, 1), (총리, NNG, 1, 2), (세대, NNG, 4, ...","[총리, 세대, 직업]"


In [36]:
target_2_df_cnt = list(target_2_df.loc[:, 'text_class_filter'])
target_2_df_cnt = [data for inner_list in target_2_df_cnt for data in inner_list]
target_2_df_cnt = word_count(target_2_df_cnt)
print(len(target_2_df_cnt))
target_2_df_cnt[:30]

1665


[('대통령', 129),
 ('북한', 37),
 ('정상', 31),
 ('김정은', 28),
 ('회담', 27),
 ('국회', 27),
 ('후보', 25),
 ('대표', 24),
 ('민주', 23),
 ('정치', 22),
 ('정부', 19),
 ('미국', 19),
 ('회의', 19),
 ('대하다', 17),
 ('남북', 17),
 ('발표', 17),
 ('총리', 17),
 ('강화', 16),
 ('경제', 16),
 ('관련', 15),
 ('평화', 14),
 ('위하다', 14),
 ('북미', 14),
 ('정책', 14),
 ('평양', 14),
 ('뉴스', 14),
 ('국민', 13),
 ('선거', 13),
 ('파크', 12),
 ('가능', 12)]

In [37]:
target_2_df = pd.DataFrame(target_2_df_cnt,columns=['word','cnt'])
target_2_df

,word,cnt
0,대통령,129
1,북한,37
2,정상,31
3,김정은,28
4,회담,27
...,...,...
1660,세대,1
1661,직업,1
1662,금감원장,1
1663,사표,1


In [38]:
target_2_drop_df = target_2_df[target_2_df['cnt']>9]
target_2_drop_df

,word,cnt
0,대통령,129
1,북한,37
2,정상,31
3,김정은,28
4,회담,27
5,국회,27
6,후보,25
7,대표,24
8,민주,23
9,정치,22


In [39]:
target_2_df.to_csv('target_2_df.csv', encoding='utf-8-sig', index=False)
target_2_drop_df.to_csv('target_2_drop_df.csv', encoding='utf-8-sig', index=False)

### (4) Target 3 (사회)

In [42]:
target_3_df['cleaned_text'] = target_3_df['text'].apply(preprocess_text)
target_3_df['text_class'] = target_3_df['cleaned_text'].apply(part_classification)
target_3_df['text_class_filter'] = part_filtering(target_3_df, 'text_class')
target_3_df

/tmp/ipykernel_364431/39756970.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_3_df['cleaned_text'] = target_3_df['text'].apply(preprocess_text)
/tmp/ipykernel_364431/39756970.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_3_df['text_class'] = target_3_df['cleaned_text'].apply(part_classification)
100%|██████████| 598/598 [00:00<00:00, 17421.40it/s]
/tmp/ipykernel_364431/39756970.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usin

,ID,text,target,cleaned_text,text_class,text_class_filter
15,aug15,'야구장에서 암표를 팔아 적발되고 기초수급자라는 타인의 인적사항을 도용했다.',3,야구장에서 암표를 팔아 적발되고 기초수급자라는 타인의 인적사항을 도용했다,"[(야구장, NNG, 0, 3), (에서, JKB, 3, 2), (암표, NNG, ...","[야구장, 암표, 적발, 기초, 수급자, 타인, 사항, 도용]"
16,aug16,'한전 감사위원에 이성한 전 경찰청장과 조전혁 전 의원',3,한전 감사위원에 이성한 전 경찰청장과 조전혁 전 의원,"[(한전, NNP, 0, 2), (감사, NNG, 3, 2), (위원, NNG, 5...","[한전, 감사, 위원, 이성한, 경찰청장, 조전혁, 의원]"
20,aug20,박정훈이 SBS 사장이 되어 한국방송협회장에 취임했다.,3,박정훈이 SBS 사장이 되어 한국방송협회장에 취임했다,"[(박정훈, NNP, 0, 3), (이, JKS, 3, 1), (SBS, SL, 5...","[박정훈, 사장, 한국방송, 협회장, 취임]"
27,aug27,울산MBC의 사장으로 최병륜이 있습니다.,3,울산MBC의 사장으로 최병륜이 있습니다,"[(울산, NNP, 0, 2), (MBC, SL, 2, 3), (의, JKG, 5,...","[울산, 사장, 최병륜]"
29,aug29,울산중부새일센터의 직업교육 프로그램을 통해 경력단절여성 19명을 취업시키고자 했다.,3,울산중부새일센터의 직업교육 프로그램을 통해 경력단절여성 19명을 취업시키고자 했다,"[(울산, NNP, 0, 2), (중부, NNG, 2, 2), (새, MM, 4, ...","[울산, 중부, 센터, 직업, 교육, 프로그램, 통하다, 경력, 단절, 여성, 취업]"
...,...,...,...,...,...,...
4180,ynat-v1_train_02756,항소심 선고 출석하는 이명박,3,항소심 선고 출석하는 이명박,"[(항소심, NNG, 0, 3), (선고, NNG, 4, 2), (출석, NNG, ...","[항소심, 선고, 출석, 이명박]"
4185,ynat-v1_train_02761,게시판 EBS대전방송 인재 양성 위한 업무협약,3,게시판 EBS대전방송 인재 양성 위한 업무협약,"[(게시판, NNG, 0, 3), (EBS, SL, 4, 3), (대전방송, NNP...","[게시판, 대전방송, 인재, 양성, 위하다, 업무, 협약]"
4189,ynat-v1_train_02765,LG U+ 서울 곳곳 나노객 증가…귀가·출근 거리,3,LG U 서울 곳곳 나노객 증가 귀가 출근 거리,"[(LG, SL, 0, 2), (U, SL, 3, 1), (서울, NNP, 5, 2...","[서울, 곳곳, 나노, 증가, 귀가, 출근, 거리]"
4208,ynat-v1_train_02788,남원소식 춘계학예단장 모집,3,남원소식 춘계학예단장 모집,"[(남원, NNP, 0, 2), (소식, NNG, 2, 2), (춘계, NNG, 5...","[남원, 소식, 춘계, 학예, 단장, 모집]"


In [43]:
target_3_df_cnt = list(target_3_df.loc[:, 'text_class_filter'])
target_3_df_cnt = [data for inner_list in target_3_df_cnt for data in inner_list]
target_3_df_cnt = word_count(target_3_df_cnt)
print(len(target_3_df_cnt))
target_3_df_cnt[:30]

1652


[('학교', 38),
 ('교육', 32),
 ('서울', 22),
 ('대하다', 19),
 ('학생', 18),
 ('경찰', 18),
 ('게시판', 18),
 ('사장', 17),
 ('공무원', 17),
 ('조사', 16),
 ('방송', 15),
 ('수업', 15),
 ('퍼센트', 15),
 ('지원', 14),
 ('코로나', 14),
 ('기자', 14),
 ('위원회', 13),
 ('올해', 13),
 ('언론', 13),
 ('대학', 13),
 ('위하다', 13),
 ('사회', 13),
 ('충북', 12),
 ('교통', 12),
 ('세계', 12),
 ('한국', 12),
 ('교육청', 12),
 ('노조', 11),
 ('디지털', 11),
 ('채용', 11)]

In [44]:
target_3_df = pd.DataFrame(target_3_df_cnt,columns=['word','cnt'])
target_3_df

,word,cnt
0,학교,38
1,교육,32
2,서울,22
3,대하다,19
4,학생,18
...,...,...
1647,출근,1
1648,남원,1
1649,춘계,1
1650,학예,1


In [45]:
target_3_drop_df = target_3_df[target_3_df['cnt']>9]
target_3_drop_df

,word,cnt
0,학교,38
1,교육,32
2,서울,22
3,대하다,19
4,학생,18
5,경찰,18
6,게시판,18
7,사장,17
8,공무원,17
9,조사,16


In [46]:
target_3_df.to_csv('target_3_df.csv', encoding='utf-8-sig', index=False)
target_3_drop_df.to_csv('target_3_drop_df.csv', encoding='utf-8-sig', index=False)

### (5) Target 4 (IT 과학)

In [47]:
target_4_df['cleaned_text'] = target_4_df['text'].apply(preprocess_text)
target_4_df['text_class'] = target_4_df['cleaned_text'].apply(part_classification)
target_4_df['text_class_filter'] = part_filtering(target_4_df, 'text_class')
target_4_df

/tmp/ipykernel_364431/2094526484.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_4_df['cleaned_text'] = target_4_df['text'].apply(preprocess_text)
/tmp/ipykernel_364431/2094526484.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_4_df['text_class'] = target_4_df['cleaned_text'].apply(part_classification)
100%|██████████| 567/567 [00:00<00:00, 17080.14it/s]
/tmp/ipykernel_364431/2094526484.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Tr

,ID,text,target,cleaned_text,text_class,text_class_filter
4,aug4,'네이버 유튜버의 지하철 2회 방송은 문화체육부로부터 2편 모두 호평을 받았다.',4,네이버 유튜버의 지하철 2회 방송은 문화체육부로부터 2편 모두 호평을 받았다,"[(네이버, NNP, 0, 3), (유튜버, NNP, 4, 3), (의, JKG, ...","[네이버, 유튜버, 지하철, 방송, 문화, 체육부, 호평]"
8,aug8,KT가 'Be Y 폰 2'를 단독으로 출시했다.,4,KT가 Be Y 폰 2 를 단독으로 출시했다,"[(KT, SL, 0, 2), (가, JKS, 2, 1), (Be, SL, 4, 2...","[단독, 출시]"
12,aug12,SK텔레콤은 충청 지역에서 발생한 폭우로 인한 통신망 문제를 신속히 복구하고 있습니다.,4,SK텔레콤은 충청 지역에서 발생한 폭우로 인한 통신망 문제를 신속히 복구하고 있습니다,"[(SK, SL, 0, 2), (텔레콤, NNG, 2, 3), (은, JX, 5, ...","[텔레콤, 충청, 지역, 발생, 폭우, 인하다, 통신망, 문제, 복구]"
13,aug13,"'삼성 갤럭시 A8가 5일 출시되며, KT는 오늘부터 예약판매를 시작합니다.'",4,삼성 갤럭시 A8가 5일 출시되며 KT는 오늘부터 예약판매를 시작합니다,"[(삼성, NNP, 0, 2), (갤럭시, NNP, 3, 3), (A, SL, 7,...","[삼성, 갤럭시, 출시, 오늘, 예약, 판매, 시작]"
23,aug23,'국가 차원에서 지능정보사회에 대비하기 위한 종합 대책을 수립',4,국가 차원에서 지능정보사회에 대비하기 위한 종합 대책을 수립,"[(국가, NNG, 0, 2), (차원, NNG, 3, 2), (에서, JKB, 5...","[국가, 차원, 지능, 정보, 사회, 대비, 위하다, 대책, 수립]"
...,...,...,...,...,...,...
4202,ynat-v1_train_02781,화합물 몸 안에서 짝퉁 단백질 작용하는 법 새로 발견,4,화합물 몸 안에서 짝퉁 단백질 작용하는 법 새로 발견,"[(화합물, NNG, 0, 3), (몸, NNG, 4, 1), (안, NNG, 6,...","[화합물, 짝퉁, 단백질, 작용, 발견]"
4207,ynat-v1_train_02787,아이폰 노바 라이프·미니어패스·아이폰10 결합상품 출시,4,아이폰 노바 라이프 미니어패스 아이폰10 결합상품 출시,"[(아이폰, NNP, 0, 3), (노바, NNP, 4, 2), (라이프, NNP,...","[아이폰, 노바, 라이프, 미니어패스, 아이폰, 결합, 상품, 출시]"
4215,ynat-v1_train_02796,삼성 갤럭시S9 정식 출시 첫 주말 이통시장 잠잠,4,삼성 갤럭시S9 정식 출시 첫 주말 이통시장 잠잠,"[(삼성, NNP, 0, 2), (갤럭시S, NNP, 3, 4), (9, SN, 7...","[삼성, 갤럭시S, 정식, 출시, 주말, 이통, 시장, 잠자다]"
4216,ynat-v1_train_02797,텔레그램 한인 등 아시아서 2시간 다운…서비스 정상화종료2보,4,텔레그램 한인 등 아시아서 2시간 다운 서비스 정상화종료2보,"[(텔레그램, NNP, 0, 4), (한인, NNG, 5, 2), (등, NNB, ...","[텔레그램, 한인, 아시아, 시간, 서비스, 정상, 종료]"


In [48]:
target_4_df_cnt = list(target_4_df.loc[:, 'text_class_filter'])
target_4_df_cnt = [data for inner_list in target_4_df_cnt for data in inner_list]
target_4_df_cnt = word_count(target_4_df_cnt)
print(len(target_4_df_cnt))
target_4_df_cnt[:30]

1444


[('출시', 57),
 ('갤럭시', 43),
 ('아이폰', 40),
 ('개발', 40),
 ('삼성', 31),
 ('서비스', 29),
 ('스마트폰', 29),
 ('네이버', 26),
 ('기술', 26),
 ('시작', 23),
 ('애플', 19),
 ('문제', 18),
 ('인공', 18),
 ('판매', 17),
 ('플랫폼', 17),
 ('연구', 16),
 ('구글', 16),
 ('지능', 15),
 ('보안', 15),
 ('세계', 15),
 ('게시판', 14),
 ('정보', 13),
 ('지원', 13),
 ('시스템', 13),
 ('국내', 13),
 ('인터넷', 13),
 ('시장', 13),
 ('예약', 12),
 ('위하다', 12),
 ('스마트', 12)]

In [49]:
target_4_df = pd.DataFrame(target_4_df_cnt,columns=['word','cnt'])
target_4_df

,word,cnt
0,출시,57
1,갤럭시,43
2,아이폰,40
3,개발,40
4,삼성,31
...,...,...
1439,잠자다,1
1440,텔레그램,1
1441,한인,1
1442,아시아,1


In [50]:
target_4_drop_df = target_4_df[target_4_df['cnt']>9]
target_4_drop_df

,word,cnt
0,출시,57
1,갤럭시,43
2,아이폰,40
3,개발,40
4,삼성,31
5,서비스,29
6,스마트폰,29
7,네이버,26
8,기술,26
9,시작,23


In [51]:
target_4_df.to_csv('target_4_df.csv', encoding='utf-8-sig', index=False)
target_4_drop_df.to_csv('target_4_drop_df.csv', encoding='utf-8-sig', index=False)

### (6) Target 5 (경제)

In [52]:
target_5_df['cleaned_text'] = target_5_df['text'].apply(preprocess_text)
target_5_df['text_class'] = target_5_df['cleaned_text'].apply(part_classification)
target_5_df['text_class_filter'] = part_filtering(target_5_df, 'text_class')
target_5_df

/tmp/ipykernel_364431/3205921087.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_5_df['cleaned_text'] = target_5_df['text'].apply(preprocess_text)
/tmp/ipykernel_364431/3205921087.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_5_df['text_class'] = target_5_df['cleaned_text'].apply(part_classification)
100%|██████████| 615/615 [00:00<00:00, 17691.54it/s]
/tmp/ipykernel_364431/3205921087.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Tr

,ID,text,target,cleaned_text,text_class,text_class_filter
0,aug0,'두산밥캣 맥스 컨트롤 2019년 혁신제품으로 골드상을 수상하였다.',5,두산밥캣 맥스 컨트롤 2019년 혁신제품으로 골드상을 수상하였다,"[(두산, NNP, 0, 2), (밥캣, NNP, 2, 2), (맥스, NNP, 5...","[두산, 밥캣, 맥스, 컨트롤, 혁신, 제품, 골드, 수상]"
1,aug1,'삼성전자 특징주가 엿새 만에 약세로 전환되었습니다.',5,삼성전자 특징주가 엿새 만에 약세로 전환되었습니다,"[(삼성전자, NNP, 0, 4), (특징, NNG, 5, 2), (주, NNG, ...","[삼성전자, 특징, 엿새, 약세, 전환]"
22,aug22,"'32만 명의 관광객 중 39%가 감소하고 있으며, 이는 사우디의 복귀로 인한 영향...",5,32만 명의 관광객 중 39퍼센트가 감소하고 있으며 이는 사우디의 복귀로 인한 영향...,"[(32, SN, 0, 2), (만, NR, 2, 1), (명, NNB, 4, 1)...","[관광객, 퍼센트, 감소, 사우디, 복귀, 인하다, 영향]"
51,aug52,SK 주식회사에 대해 공모된 자금이 총 12조 원으로 순매출되었습니다.,5,SK 주식회사에 대해 공모된 자금이 총 12조 원으로 순매출되었습니다,"[(SK, SL, 0, 2), (주식회사, NNG, 3, 4), (에, JKB, 7...","[주식회사, 대하다, 공모, 자금, 매출]"
53,aug54,그랜드백화점의 토즈 자산을 평가하여 정리할 계획을 세우고 있다.,5,그랜드백화점의 토즈 자산을 평가하여 정리할 계획을 세우고 있다,"[(그랜드백화점, NNP, 0, 6), (의, JKG, 6, 1), (토즈, NNP...","[그랜드백화점, 토즈, 자산, 평가, 정리, 계획, 세우다]"
...,...,...,...,...,...,...
4181,ynat-v1_train_02757,여권번호 안심예약 3중복예약…전체 62% 증가,5,여권번호 안심예약 3중복예약 전체 62퍼센트 증가,"[(여권, NNG, 0, 2), (번호, NNG, 2, 2), (안심, NNG, 5...","[여권, 번호, 안심, 예약, 중복, 예약, 전체, 퍼센트, 증가]"
4192,ynat-v1_train_02769,두산밥캣 맥스 컨트롤 2019 혁신제품 골드 수상,5,두산밥캣 맥스 컨트롤 2019 혁신제품 골드 수상,"[(두산, NNP, 0, 2), (밥캣, NNP, 2, 2), (맥스, NNP, 5...","[두산, 밥캣, 맥스, 컨트롤, 제품, 골드, 수상]"
4194,ynat-v1_train_02772,원달러 환율 하락해…1220선 아프리카 경기부양 영향,5,원달러 환율 하락해 1220선 아프리카 경기부양 영향,"[(원, NNG, 0, 1), (달러, NNG, 1, 2), (환율, NNG, 4,...","[달러, 환율, 하락, 아프리카, 경기, 부양, 영향]"
4199,ynat-v1_train_02778,당국 핀테크위크 참여할 레그테크·섭테크 기업 모집,5,당국 핀테크위크 참여할 레그테크 섭테크 기업 모집,"[(당국, NNG, 0, 2), (핀테크, NNG, 3, 3), (위크, NNG, ...","[당국, 핀테크, 위크, 참여, 레그테크, 섭테크, 기업, 모집]"


In [53]:
target_5_df_cnt = list(target_5_df.loc[:, 'text_class_filter'])
target_5_df_cnt = [data for inner_list in target_5_df_cnt for data in inner_list]
target_5_df_cnt = word_count(target_5_df_cnt)
print(len(target_5_df_cnt))
target_5_df_cnt[:30]

1540


[('퍼센트', 111),
 ('투자', 46),
 ('상승', 46),
 ('분기', 38),
 ('시장', 33),
 ('코스피', 31),
 ('주식', 31),
 ('증가', 31),
 ('금융', 26),
 ('하락', 25),
 ('은행', 25),
 ('달러', 23),
 ('대출', 23),
 ('이익', 21),
 ('자금', 19),
 ('주택', 19),
 ('증권', 19),
 ('판매', 19),
 ('한국', 18),
 ('전망', 17),
 ('지원', 17),
 ('영업', 17),
 ('대하다', 16),
 ('그래픽', 16),
 ('규모', 16),
 ('실적', 16),
 ('목표', 16),
 ('경제', 15),
 ('펀드', 15),
 ('협력', 15)]

In [55]:
target_5_df = pd.DataFrame(target_5_df_cnt,columns=['word','cnt'])
target_5_df

,word,cnt
0,퍼센트,111
1,투자,46
2,상승,46
3,분기,38
4,시장,33
...,...,...
1535,아프리카,1
1536,핀테크,1
1537,위크,1
1538,레그테크,1


In [56]:
target_5_drop_df = target_5_df[target_5_df['cnt']>9]
target_5_drop_df

,word,cnt
0,퍼센트,111
1,투자,46
2,상승,46
3,분기,38
4,시장,33
5,코스피,31
6,주식,31
7,증가,31
8,금융,26
9,하락,25


In [57]:
target_5_df.to_csv('target_5_df.csv', encoding='utf-8-sig', index=False)
target_5_drop_df.to_csv('target_5_drop_df.csv', encoding='utf-8-sig', index=False)

### (7) Target 6 (세계)

In [58]:
target_6_df['cleaned_text'] = target_6_df['text'].apply(preprocess_text)
target_6_df['text_class'] = target_6_df['cleaned_text'].apply(part_classification)
target_6_df['text_class_filter'] = part_filtering(target_6_df, 'text_class')
target_6_df

/tmp/ipykernel_364431/3649243629.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_6_df['cleaned_text'] = target_6_df['text'].apply(preprocess_text)
/tmp/ipykernel_364431/3649243629.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_6_df['text_class'] = target_6_df['cleaned_text'].apply(part_classification)
100%|██████████| 633/633 [00:00<00:00, 17376.98it/s]
/tmp/ipykernel_364431/3649243629.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Tr

,ID,text,target,cleaned_text,text_class,text_class_filter
28,aug28,"'한국 언론, 대통령의 역사 문제 언급을 자제하고 일본에 대한 비난을 억제하는 종합...",6,한국 언론 대통령의 역사 문제 언급을 자제하고 일본에 대한 비난을 억제하는 종합적인...,"[(한국, NNP, 0, 2), (언론, NNG, 3, 2), (대통령, NNG, ...","[한국, 언론, 대통령, 역사, 문제, 언급, 자제, 일본, 대하다, 비난, 억제,..."
34,aug34,'북한의 발사체에 대해 미·일 당국이 통화로 협력한 사실이 확인되었다.',6,북한의 발사체에 대해 미 일 당국이 통화로 협력한 사실이 확인되었다,"[(북한, NNP, 0, 2), (의, JKG, 2, 1), (발사체, NNP, 4...","[북한, 발사체, 대하다, 당국, 통화, 협력, 사실, 확인]"
40,aug40,'드례크루즈 미사일 친이시설 목표가 변경되었다.',6,드례크루즈 미사일 친이시설 목표가 변경되었다,"[(드례, NNP, 0, 2), (크루즈, NNG, 2, 3), (미사일, NNG,...","[드례, 크루즈, 미사일, 시설, 목표, 변경]"
43,aug43,'日도쿄 주택가 공원 연못에서 시신이 토막나서 발견되었다.',6,도쿄 주택가 공원 연못에서 시신이 토막나서 발견되었다,"[(도쿄, NNP, 0, 2), (주택가, NNG, 3, 3), (공원, NNG, ...","[도쿄, 주택가, 공원, 연못, 시신, 토막, 발견]"
45,aug46,'폴란드에서 이란을 겨냥한 미국 주도의 중동 문제 회의가 시작되었다.',6,폴란드에서 이란을 겨냥한 미국 주도의 중동 문제 회의가 시작되었다,"[(폴란드, NNP, 0, 3), (에서, JKB, 3, 2), (이란, NNP, ...","[폴란드, 이란, 겨냥, 미국, 주도, 중동, 문제, 회의, 시작]"
...,...,...,...,...,...,...
4169,ynat-v1_train_02742,美가 프리다칼로 사상 이해 안돼…벌점 인정 주멕시코 美대사,6,가 프리다칼로 사상 이해 안돼 벌점 인정 주멕시코 대사,"[(가, NNG, 0, 1), (프리, NNG, 2, 2), (다, MAG, 4, ...","[프리, 사상, 이해, 벌점, 인정, 멕시코, 대사]"
4170,ynat-v1_train_02743,美 묘지 묘막식 2년만에 일상화…8주주의·종도리문서 새로,6,묘지 묘막식 2년만에 일상화 8주주의 종도리문서 새로,"[(묘지, NNG, 0, 2), (묘막식, NNG, 3, 3), (2, SN, 7,...","[묘지, 묘막식, 일상, 주의, 종도리, 문서]"
4178,ynat-v1_train_02754,중동 유일 UAE 아부다비 北식당 옥류관 영업중단,6,중동 유일 UAE 아부다비 식당 옥류관 영업중단,"[(중동, NNP, 0, 2), (유일, NNG, 3, 2), (UAE, SL, 6...","[중동, 유일, 아부다비, 식당, 옥류관, 영업, 중단]"
4191,ynat-v1_train_02768,트럼프 또 런던시장에 악감정 표출 그는 재앙…새 시장 필요해,6,트럼프 또 런던시장에 악감정 표출 그는 재앙 새 시장 필요해,"[(트럼프, NNP, 0, 3), (또, MAG, 4, 1), (런던, NNP, 6...","[트럼프, 런던, 시장, 악감정, 표출, 재앙, 시장, 필요]"


In [59]:
target_6_df_cnt = list(target_6_df.loc[:, 'text_class_filter'])
target_6_df_cnt = [data for inner_list in target_6_df_cnt for data in inner_list]
target_6_df_cnt = word_count(target_6_df_cnt)
print(len(target_6_df_cnt))
target_6_df_cnt[:30]

1841


[('이란', 75),
 ('미국', 49),
 ('중국', 41),
 ('트럼프', 37),
 ('북한', 32),
 ('대하다', 31),
 ('대통령', 30),
 ('제재', 25),
 ('홍콩', 23),
 ('총리', 22),
 ('정부', 22),
 ('한국', 21),
 ('미사일', 20),
 ('인도', 20),
 ('이스라엘', 20),
 ('일본', 19),
 ('프랑스', 19),
 ('유엔', 19),
 ('러시아', 18),
 ('테러', 18),
 ('의회', 18),
 ('반대', 17),
 ('중단', 16),
 ('퍼센트', 16),
 ('시위', 16),
 ('합의', 16),
 ('공격', 16),
 ('위하다', 15),
 ('경찰', 15),
 ('세계', 15)]

In [60]:
target_6_df = pd.DataFrame(target_6_df_cnt,columns=['word','cnt'])
target_6_df

,word,cnt
0,이란,75
1,미국,49
2,중국,41
3,트럼프,37
4,북한,32
...,...,...
1836,표출,1
1837,폭스뉴스,1
1838,앵커,1
1839,매기다,1


In [61]:
target_6_drop_df = target_6_df[target_6_df['cnt']>9]
target_6_drop_df

,word,cnt
0,이란,75
1,미국,49
2,중국,41
3,트럼프,37
4,북한,32
5,대하다,31
6,대통령,30
7,제재,25
8,홍콩,23
9,총리,22


In [62]:
target_6_df.to_csv('target_6_df.csv', encoding='utf-8-sig', index=False)
target_6_drop_df.to_csv('target_6_drop_df.csv', encoding='utf-8-sig', index=False)

## 요기껀 안쓰는거..

In [ ]:
# 각 클러스터의 데이터 개수 계산
label_counts = df['target'].value_counts().sort_index()

# 시각화
plt.figure(figsize=(10, 6))
sns.barplot(x=label_counts.index, y=label_counts.values, palette='husl')

# 그래프에 레이블 추가
for i, count in enumerate(label_counts.values):
    plt.text(i, count, str(count), ha='center', va='bottom')

# 그래프 설정
plt.title('Number of Data Points in Each Label')
plt.xlabel('label')
plt.ylabel('Number of Data Points')
plt.xticks(label_counts.index)
plt.show()

In [ ]:
kiwi = Kiwi()

In [ ]:
# 품사 분류
def part_classification(text):
    """
    텍스트의 품사를 태깅(분류)하는 함수

    Args:
        text: 데이터셋의 'text'열의 데이터

    Returns:
        raw_part_class: 각 텍스트의 토큰에 대한 품사 정보
    """
    raw_part_class = kiwi.tokenize(text, normalize_coda=False)
    return raw_part_class

In [ ]:
# 주요 품사 필터링 (명사, 동사, 형용사, 어근)
MAIN_PART = ['NNG', 'NNB', 'NNP', 'VV', 'VA', 'XR'] # 필터링할 주요 품사
VERB_ADJ_PART = ['VV', 'VA'] # 동사, 형용사

def part_filtering(text):
    """
    텍스트에서 명사, 동사, 형용사, 어근 품사만 추출하여 필터링하는 함수
    """
    tokens = kiwi.tokenize(text, normalize_coda=False)
    class_filter = [
        token.form + '다' if token.tag in VERB_ADJ_PART else token.form
        for token in tokens
        if token.tag in MAIN_PART and len(token.form) > 1
    ]
    return ' '.join(class_filter)  # 조사를 제거한 단어를 공백으로 연결

df['filtered_text'] = df['text'].apply(part_filtering)
df.head()

In [ ]:
# TF-IDF 벡터화
tfidf_vect = TfidfVectorizer(ngram_range=(1, 2), max_features=5000)  # 유용한 feature 제한
tfidf_matrix = tfidf_vect.fit_transform(df['filtered_text'])

In [ ]:
# K-Means 클러스터링 (카테고리가 7개라고 가정)
from sklearn.cluster import KMeans

num_clusters = 7
kmeans = KMeans(n_clusters=num_clusters, max_iter=10000, random_state=42)
df['cluster'] = kmeans.fit_predict(tfidf_matrix)

In [ ]:
# 각 클러스터별로 주요 단어 추출
def get_top_keywords(tfidf_matrix, cluster_labels, n_terms=50):
    df_keywords = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vect.get_feature_names_out())
    df_keywords['cluster'] = cluster_labels
    top_keywords = {}

    for cluster in range(num_clusters):
        top_keywords[cluster] = (
            df_keywords[df_keywords['cluster'] == cluster]
            .drop('cluster', axis=1)
            .mean()
            .sort_values(ascending=False)
            .head(n_terms)
            .index.tolist()
        )
    return top_keywords


In [ ]:
# 각 클러스터별 주요 키워드 확인
top_keywords = get_top_keywords(tfidf_matrix, df['cluster'])
print("클러스터별 주요 키워드:")
for cluster, keywords in top_keywords.items():
    print(f"Cluster {cluster}: {keywords}")

In [ ]:
# 각 클러스터의 주요 단어 빈도 계산
def get_cluster_word_counts(tfidf_matrix, cluster_labels):
    """
    각 클러스터의 단어 빈도를 계산하는 함수

    Args:
        tfidf_matrix: TF-IDF로 벡터화된 텍스트 행렬
        cluster_labels: 각 텍스트의 클러스터 라벨

    Returns:
        cluster_word_counts: 각 클러스터별 단어 빈도를 저장한 리스트
    """
    # TF-IDF 행렬을 데이터프레임으로 변환하여 단어 이름을 매칭
    tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vect.get_feature_names_out())
    tfidf_df['cluster'] = cluster_labels
    
    # 각 클러스터별 단어 빈도 계산
    cluster_word_counts = {}
    for cluster in range(num_clusters):
        cluster_df = tfidf_df[tfidf_df['cluster'] == cluster].drop('cluster', axis=1)
        word_counts = cluster_df.sum(axis=0).sort_values(ascending=False)
        cluster_word_counts[cluster] = word_counts[word_counts > 0].index.tolist()
        
    return cluster_word_counts

# 클러스터별 단어 빈도 계산
cluster_word_counts = get_cluster_word_counts(tfidf_matrix, df['cluster'])

In [ ]:
# 클러스터 간 단어 중복 및 고유 단어 확인
def get_unique_and_common_words(cluster_word_counts):
    """
    클러스터별 고유 단어와 중복 단어를 구분하는 함수

    Args:
        cluster_word_counts: 각 클러스터별 단어 리스트

    Returns:
        unique_words: 클러스터별 고유 단어 딕셔너리
        common_words: 모든 클러스터에 중복된 단어 리스트
    """
    # 모든 클러스터의 단어들을 합집합으로 모아 공통 단어와 고유 단어 계산
    all_words = set().union(*cluster_word_counts.values())
    common_words = set(all_words)
    
    unique_words = {}
    for cluster, words in cluster_word_counts.items():
        other_clusters_words = set().union(
            *[cluster_word_counts[other] for other in cluster_word_counts if other != cluster]
        )
        unique_words[cluster] = list(set(words) - other_clusters_words)
        common_words &= set(words)
    
    return unique_words, list(common_words)

# 각 클러스터별 고유 단어와 모든 클러스터에 중복되는 단어 확인
unique_words, common_words = get_unique_and_common_words(cluster_word_counts)

In [ ]:
# 결과 출력
print("클러스터별 고유 단어:")
for cluster, words in unique_words.items():
    print(f"Cluster {cluster}: {words[:30]}")  # 상위 10개의 고유 단어만 출력

print("\n모든 클러스터에 중복된 단어:")
print(common_words[:30])  # 상위 10개의 중복 단어만 출력